<a href="https://colab.research.google.com/github/abdulsamadkhan/LLM/blob/main/fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FineTunning (PyTorch)


The objective is to fine-tune a BERT (bert-base-uncased) model for a specific task.

1. Step 1: Loading the Model
   - The first step involves loading a pretrained BERT model (bert-base-uncased).
   - This model serves as the starting point and provides the underlying architecture and pre-trained weights.

2. Step 2: Hugging Face Dataset Loading
   - The next step is to load the dataset using Hugging Face's dataset library.
   - Hugging Face provides a convenient way to access and manage various datasets for natural language processing tasks.

3. Step 3: Preprocessing the Dataset using the Tokenizer
   - After loading the dataset, it needs to be preprocessed to match the input format expected by the BERT model.
   - This is done by applying the BERT tokenizer, which tokenizes the text into subword units and adds special tokens such as [CLS] and [SEP].

4. Step 4: Loading the Dataset using Hugging Face DataLoader
   - Once the dataset is preprocessed, it needs to be loaded into a DataLoader provided by Hugging Face.
   - The DataLoader efficiently handles batching, shuffling, and other data loading operations required for training the model.

5. Step 5: Finetuning the Model using a Training Loop
   - The main objective is to finetune the BERT model for the specific task.
   - This is achieved by training the model on the loaded dataset using a training loop.
   - During each iteration of the loop, a batch of input data is fed to the model, and the model's parameters are updated based on the computed gradients.

6. Step 6: Finetuning using the Trainer Class
   - To simplify the finetuning process, Hugging Face provides the Trainer class.
   - The Trainer class encapsulates the training loop and provides additional functionalities such as handling optimization, learning rate scheduling, and logging.

Overall Objective:
The objective is to adapt the pretrained BERT model to the specific task by finetuning it. This involves loading the model, loading the dataset, preprocessing the dataset using the tokenizer, loading the preprocessed dataset into a DataLoader, and then training the model on the dataset using a training loop or the Trainer class. The goal is to improve the model's performance on the specific task by leveraging the pre-trained BERT architecture and weights and adapting them through the finetuning process.

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 115.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

# Step 1: Loading the Model BERT (bert-base-uncased)
BERT is a transformers model pretrained on a large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labeling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts. More precisely, it was pretrained with two objectives:

* Masked language modeling (MLM): taking a sentence, the model randomly masks 15% of the words in the input then run the entire masked sentence through the model and has to predict the masked words. This is different from traditional recurrent neural networks (RNNs) that usually see the words one after the other, or from autoregressive models like GPT which internally masks the future tokens. It allows the model to learn a bidirectional representation of the sentence.
* Next sentence prediction (NSP): the models concatenates two masked sentences as inputs during pretraining. Sometimes they correspond to sentences that were next to each other in the original text, sometimes not. The model then has to predict if the two sentences were following each other or not.

This way, the model learns an inner representation of the English language that can then be used to extract features useful for downstream tasks.
https://huggingface.co/bert-base-uncased

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

#print the batch
print(batch)
#printing the structure of the model
print(model)
#generating the model output
output = model(**batch)
#
# This is new
#batch["labels"] = torch.tensor([1, 1])
## Now we can use the model to make predictions on the batch # Predicting on the batch
#outputs = model(**batch)
print(output)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2023,  2607,  2003,  6429,   999,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x Be

#Step 2: Hugging Face Dataset Loading
<a href="https://huggingface.co/datasets/glue/viewer/mrpc/test">MRPC</a> <a href="https://paperswithcode.com/dataset/mrpc">dataset</a>! This is one of the 10 datasets composing the GLUE <a href="https://gluebenchmark.com/"> GLUE</a>

 benchmark, which is an academic benchmark that is used to measure the performance of ML models across 10 different text classification tasks.

In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [5]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

# Step 3: Preprocessing the Dataset using the Tokenizer
loading the tokenizer
A tokenizer is in charge of preparing the inputs for a model.

In [6]:

from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
#tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])
#print("First Sentence in the training data",raw_datasets["train"]["sentence1"][0])
#print(tokenized_sentences_1['input_ids'][0])

In [7]:
# apply to tokenizer to simple sentence
input_string= "I am taking NLP class at university."
inputs = tokenizer(input_string)
print("Ecoding to this model for simple sentence: ",inputs)
tokens = tokenizer.tokenize(input_string)
print("tokens in the input string",tokens)
text=tokenizer.convert_ids_to_tokens(inputs["input_ids"])
print("Input text to model",text)

Ecoding to this model for simple sentence:  {'input_ids': [101, 1045, 2572, 2635, 17953, 2361, 2465, 2012, 2118, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
tokens in the input string ['i', 'am', 'taking', 'nl', '##p', 'class', 'at', 'university', '.']
Input text to model ['[CLS]', 'i', 'am', 'taking', 'nl', '##p', 'class', 'at', 'university', '.', '[SEP]']


In [8]:
# Checking some options with tokenizer
input_string= "I am taking NLP class at university."

encoded_input = tokenizer.encode_plus(input_string, max_length=20, pad_to_max_length=True, truncation=True)

input_ids = encoded_input["input_ids"]
attention_mask = encoded_input["attention_mask"]

print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)


Input IDs: [101, 1045, 2572, 2635, 17953, 2361, 2465, 2012, 2118, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
# We can apply it to two sentences. check the token_type _ids
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

# Step 3: Preprocessing the Dataset using the Tokenizer

In [11]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

sentence1_lengths = [len(tokens) for tokens in tokenized_dataset["input_ids"]]

print("Sentence 1 Lengths:", sentence1_lengths)

print(tokenized_dataset["input_ids"][0])
# Importing necessary libraries


instance_index = 0  # Index of the instance you want to print

instance = tokenized_dataset["input_ids"][instance_index]
tokens = tokenizer.convert_ids_to_tokens(instance)

print("Instance:", instance)
print("Tokens:", tokens)


Sentence 1 Lengths: [103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,

In [12]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [13]:
tokenized_datasets = raw_datasets.map(tokenize_function)

# Access the data
for batch in tokenized_datasets['train']:
    print(batch)
    break


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0, 'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


The <a href="https://huggingface.co/docs/transformers/main_classes/data_collator">DataCollatorWithPadding</a> class from the Hugging Face's Transformers library is a utility class that helps with batch processing and padding of tokenized sequences. It is often used in conjunction with a tokenizer to prepare data for model training or evaluation.
The purpose of a data collator is to handle the differences in sequence lengths within a batch. It ensures that all sequences within a batch have the same length by padding shorter sequences with appropriate padding tokens. This is necessary because deep learning models typically require fixed-length inputs.

The DataCollatorWithPadding class, specifically, pads the sequences using the padding token provided by the tokenizer. It automatically determines the maximum sequence length within a batch and pads all sequences to match that length



In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
samples = tokenized_datasets["train"][:3]
for k, v in samples.items():
  print("key:", k)
  print("Value",v)
  print("---")

#removing sentence1, sentence2 and idx
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}

[len(x) for x in samples["input_ids"]]

key: sentence1
Value ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .']
---
key: sentence2
Value ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .", "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale ."]
---
key: label
Value [1, 0, 1]
---
key: idx
Value [0, 1, 2]
---
key: input_ids
Value [[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 

[50, 59, 47]

In [16]:
#batching process
batch = data_collator(samples)

{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([3, 59]),
 'token_type_ids': torch.Size([3, 59]),
 'attention_mask': torch.Size([3, 59]),
 'labels': torch.Size([3])}

# Putting all together in one block

In [17]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [18]:
# preparing the data for fine-tunning
#removing the columns "sentence1", "sentence2", "idx"
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
#renaming the label to "labels"
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
#brining to pytorch format
tokenized_datasets.set_format("torch")
#printing the columns in the tokenized dataset
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

# Step 4: Loading the Dataset using Hugging Face DataLoader
The <a href ="https://towardsdatascience.com/using-a-dataloader-in-hugging-face-52388f552259">DataLoader</a> class from the torch.utils.data module in PyTorch provides an iterable over a dataset, allowing you to efficiently load and process data in batches during model training or evaluation. It is commonly used to handle data loading and batching in machine learning tasks.

In [19]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [20]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 75]),
 'token_type_ids': torch.Size([8, 75]),
 'attention_mask': torch.Size([8, 75])}

In [30]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
#To print the last layer of the model and verify if there are three neurons for the output,
#you can access the classifier attribute of the model.
print("last layer structure",model.classifier)

print("weights of the last layer",model.classifier.weight)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

last layer structure Linear(in_features=768, out_features=2, bias=True)
weights of the last layer Parameter containing:
tensor([[ 0.0146,  0.0262, -0.0093,  ...,  0.0310,  0.0158, -0.0231],
        [ 0.0024, -0.0440,  0.0314,  ...,  0.0073, -0.0262,  0.0021]],
       requires_grad=True)


# inference
*  **outputs = model(batch):** We pass the batch to the model using ** to unpack the dictionary. This allows us to provide the necessary input to the model for making predictions. The model takes the input sequences, applies its architecture and weights, and produces outputs. The outputs variable will contain the results of the model's prediction.

* **print(outputs.loss, outputs.logits.shape):** We print two pieces of information from the outputs. The loss represents the computed loss value associated with the model's predictions. This loss value is used during training to guide the model's learning process. The logits are the raw output values produced by the model before applying any activation functions or probabilities. We print the shape of the logits tensor to see the dimensions of the output, which would typically correspond to the batch size and the number of labels or classes in the sequence classification task.

In [31]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

RuntimeError: ignored

In [32]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


1. By calling **get_scheduler()** with the specified arguments, you create a learning rate scheduler object **(lr_scheduler)** that will adjust the learning rate of the optimizer during the training process. The learning rate will follow a linear schedule, decreasing gradually over the training steps.
1. During training, you can call **lr_scheduler.step()** after each optimization step **(optimizer.step())** to update the learning rate according to the specified schedule. The scheduler will adjust the learning rate based on the current training step, the number of warm-up steps, and the total number of training steps

In [33]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


In [34]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

#Step 5: Finetuning the Model using a Training Loop

          Input Sequences
                |
                V
              Model
                |
                V
        Model Outputs (Predictions)
                |
                V
         Compute Loss (Criterion)
                |
                V
         Backpropagation (Compute Gradients)
                |
                V
           Update Model Weights
                |
                V
         Update Learning Rate (Scheduler)
                |
                V
        Clear Gradient Accumulation
                |
                V
           Update Progress Bar
                |
                V
     Repeat for Each Batch
                |
                V
      Repeat for Each Epoch


In [35]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [36]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8602941176470589, 'f1': 0.9038785834738617}

In [37]:
print("weights of the last layer",model.classifier.weight)

weights of the last layer Parameter containing:
tensor([[ 0.0154,  0.0280, -0.0076,  ...,  0.0325,  0.0169, -0.0245],
        [ 0.0017, -0.0458,  0.0297,  ...,  0.0057, -0.0274,  0.0035]],
       device='cuda:0', requires_grad=True)


# Step 6: Finetuning using the Trainer Class

In [1]:
pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSequenceClassification
#After instantiating a pretrained BERT model for sequence classification, you may receive warnings as the original pretrained head is discarded and a new classification head is added,
#resulting in unused and randomly initialized weights, prompting you to proceed with training
#the model.
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [19]:
import evaluate
import numpy as np
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [21]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.412519,0.833333,0.883562
2,0.517100,0.592316,0.840686,0.893617
3,0.292400,0.641461,0.862745,0.904437


TrainOutput(global_step=1377, training_loss=0.33005627844761315, metrics={'train_runtime': 206.6776, 'train_samples_per_second': 53.242, 'train_steps_per_second': 6.663, 'total_flos': 406183858377360.0, 'train_loss': 0.33005627844761315, 'epoch': 3.0})